![alt text](https://i.imgur.com/1WaY7aA.png)



---



---



# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [22]:
! pip install sqlite3

  ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [45]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [46]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [164]:
response.status_code

200

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [14]:
parameters = {"lat" :-33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [57]:
responses = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

Print the response header:

In [62]:
print (responses.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Mon, 30 Sep 2019 10:00:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Via': '1.1 vegur', 'Connection': 'Keep-Alive'}


Print the content of the response (the data that the server returned):

In [60]:
print (responses.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1569837017, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 515, \n      "risetime": 1569856322\n    }, \n    {\n      "duration": 659, \n      "risetime": 1569862029\n    }, \n    {\n      "duration": 532, \n      "risetime": 1569867932\n    }, \n    {\n      "duration": 353, \n      "risetime": 1569873917\n    }, \n    {\n      "duration": 453, \n      "risetime": 1569879775\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [64]:
print (type (responses.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [65]:
print(response.headers['Content-Type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [83]:
x = responses.json()
print(x)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1569837017, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 515, 'risetime': 1569856322}, {'duration': 659, 'risetime': 1569862029}, {'duration': 532, 'risetime': 1569867932}, {'duration': 353, 'risetime': 1569873917}, {'duration': 453, 'risetime': 1569879775}]}


What kind of object did this give us?

In [73]:
print (type (responses.json()))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [150]:
x = responses.json()

y = x ['response']
x['']

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1569837017,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 515, 'risetime': 1569856322},
  {'duration': 659, 'risetime': 1569862029},
  {'duration': 532, 'risetime': 1569867932},
  {'duration': 353, 'risetime': 1569873917},
  {'duration': 453, 'risetime': 1569879775}]}

Now extract the `risetime` strings into an array called `srisetimes`:

In [114]:
x = responses.json()
y = x ['response']
srisetime = [item ["risetime"] for item in y]
srisetime

[1569856322, 1569862029, 1569867932, 1569873917, 1569879775]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [117]:
rise = [datetime.fromtimestamp (item) for item in srisetime]
rise

[datetime.datetime(2019, 10, 1, 1, 12, 2),
 datetime.datetime(2019, 10, 1, 2, 47, 9),
 datetime.datetime(2019, 10, 1, 4, 25, 32),
 datetime.datetime(2019, 10, 1, 6, 5, 17),
 datetime.datetime(2019, 10, 1, 7, 42, 55)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [130]:
[item.strftime('%d/%m/%y %I:%M') for item in rise]

['01/10/19 01:12',
 '01/10/19 02:47',
 '01/10/19 04:25',
 '01/10/19 06:05',
 '01/10/19 07:42']

Here is an endpoint that tells us who is onboard:

In [74]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [159]:
response = requests.get("http://api.open-notify.org/astros.json")
x = response.content
y = response.json ()
y


{'people': [{'name': 'Alexey Ovchinin', 'craft': 'ISS'},
  {'name': 'Nick Hague', 'craft': 'ISS'},
  {'name': 'Christina Koch', 'craft': 'ISS'},
  {'name': 'Alexander Skvortsov', 'craft': 'ISS'},
  {'name': 'Luca Parmitano', 'craft': 'ISS'},
  {'name': 'Andrew Morgan', 'craft': 'ISS'},
  {'name': 'Oleg Skripochka', 'craft': 'ISS'},
  {'name': 'Jessica Meir', 'craft': 'ISS'},
  {'name': 'Hazzaa Ali Almansoori', 'craft': 'ISS'}],
 'number': 9,
 'message': 'success'}

In [161]:
number =  y['number']
number

9

In [158]:
z = y ["people"]
name = [item ["name"] for item in z]
name

['Alexey Ovchinin',
 'Nick Hague',
 'Christina Koch',
 'Alexander Skvortsov',
 'Luca Parmitano',
 'Andrew Morgan',
 'Oleg Skripochka',
 'Jessica Meir',
 'Hazzaa Ali Almansoori']

In [162]:
## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [188]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
     # if Status Code is 200 return false, and status code

    if response.status_code == 200:
        return [False]
        # Otherwise Return True and Status Code
    else:
        return [True, response.status_code ]
      


2. Test your response handler on some correct and incorrect API calls.

In [197]:
response = requests.get("http://api.open-notify.org/astros986.json")

if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
handleResponse(response)[0]    
#response = requests.get("http://api.open-notify.org/iss-pass.json")
#handleResponse(response)

API call failed. Resolve issue before continuing!


True

>

>

>



---



---


> > > > > > > > >© 2019 Data Science Institute of Australia


---



---

